In [20]:
require(zoo)

Carrega o pacote zoo, que fornece funções para manipulação de séries temporais e objetos de dados ordenados

In [21]:
require(data.table)

Carrega o pacote data.table, que melhora a manipulação de tabelas de dados em R, oferecendo funções eficientes e rápidas para leitura, filtragem, agregação e manipulação de grandes conjuntos de dados.

In [22]:
unemp <- fread("Ch02/data/UNRATE.csv")

Usa a função fread() do pacote data.table para ler o arquivo CSV localizado no caminho "data/UNRATE.csv" de forma rápida e eficiente. Essa função é similar à read.csv(), porém muito mais rápida, especialmente para arquivos grandes.

In [23]:
unemp[, DATE := as.Date(DATE)]

Converte a coluna DATE de unemp, que provavelmente está em formato de texto, para o formato de data do R usando as.Date(). Essa operação é feita usando a sintaxe de data.table, que permite modificar a própria tabela por referência, ou seja, sem criar uma cópia. O símbolo := indica que a operação é uma atribuição por referência dentro da tabela.

In [24]:
setkey(unemp, DATE)

Define a coluna DATE como chave da tabela unemp, o que melhora a eficiência em operações de agrupamento, ordenação e junções futuras, pois data.table utiliza essa chave para acelerar essas operações.

Definir uma coluna como chave da tabela no `data.table` em R significa ordenar a tabela por essa coluna e marcar essa coluna como "chave". A chave funciona como um índice que permite operações muito mais rápidas em manipulações de dados, como filtragens, junções (joins) e agrupamentos.

No exemplo da coluna `DATE` na tabela `unemp`:

- A tabela `unemp` será reordenada em ordem crescente pela coluna `DATE`.
- O `data.table` saberá que a coluna `DATE` é a chave, permitindo buscas e operações de filtragem baseadas nessa coluna serem executadas usando um algoritmo eficiente de busca binária.
- Com a chave definida, você pode acessar rapidamente um intervalo de datas, fazer junções com outras tabelas chaveadas por essa mesma coluna, e o desempenho será maior do que em tabelas sem chave.

Em suma, a chave em `data.table` é uma forma de otimização interna que ordena e indexa a tabela para acelerar consultas e operações que usam essa coluna como referência.
***

In [25]:
rand.unemp.idx <- sample(1:nrow(unemp), .1*nrow(unemp))

- Gera uma amostra aleatória sem reposição de índices das linhas da tabela unemp.

- 1:nrow(unemp) cria uma sequência de números de 1 até o número total de linhas da tabela.

- .1*nrow(unemp) calcula 10% do número total de linhas.

- A função sample() seleciona aleatoriamente esses índices correspondentes a 10% das linhas da tabela.

- O resultado é armazenado no vetor rand.unemp.idx, que contém os índices das linhas sorteadas.


In [26]:
rand.unemp <- unemp[-rand.unemp.idx]

- Cria uma nova tabela rand.unemp que é uma cópia da tabela unemp, porém excluindo as linhas com os índices contidos em rand.unemp.idx.

- O operador - na indexação significa "excluir" as linhas indicadas.

- Ou seja, essa linha remove os 10% de linhas sorteadas aleatoriamente para formar um subconjunto menor dos dados, contendo os 90% restantes.

***

In [30]:
high.unemp.idx <- which(unemp$UNRATE > 8)

Identifica os índices das linhas da tabela unemp onde a taxa de desemprego (UNRATE) é maior que 8%.

In [28]:
high.unemp.idx <- sample(high.unemp.idx, .5 * length(high.unemp.idx))

A partir dos índices encontrados no passo anterior, seleciona aleatoriamente metade deles (50%) para uma amostra sem reposição, simulando a remoção parcial desses casos com desemprego alto.

In [29]:
bias.unemp <- unemp[-high.unemp.idx]

Cria uma nova tabela bias.unemp removendo as linhas correspondentes aos índices sorteados, ou seja, exclui metade dos registros com desemprego maior que 8%.

A finalidade dessa operação é simular um viés na amostra, removendo parcialmente os casos de desemprego alto, o que pode representar uma situação de dados enviesados, onde certos grupos (no caso, altas taxas de desemprego) estão sub-representados por remoção ou ausência. Isso permite estudar como o viés e a falta de informações podem afetar análises estatísticas e modelos preditivos, exatamente como parte do estudo sobre tratamento e impacto de dados ausentes ou enviesados na base original.

***

In [31]:
all.dates <- seq(from = unemp$DATE[1], to = tail(unemp$DATE, 1), by = "months")

Gera uma sequência completa de datas mensais desde a primeira até a última data da tabela original unemp.

In [32]:
rand.unemp = rand.unemp[J(all.dates), roll=0]
bias.unemp = bias.unemp[J(all.dates), roll=0]

- Executa uma rolling joins (junção rolante) entre a tabela rand.unemp e a sequência completa de datas gerada, com a ajuda da função J().

- A chave para essa junção é a coluna de datas. O parâmetro roll=0 indica que a junção deve procurar uma correspondência exata para cada data; se não houver, será inserido um valor NA.

- Essa operação essencialmente "preenche" as datas ausentes na série, introduzindo explicitamente linhas com NA onde o dado não existia, estabilizando a série temporal para análise mais rigorosa.

A sintaxe do `data.table` utiliza um sistema diferente dos parênteses comuns para funções em R, especialmente para facilitar operações de subset, join e manipulação eficiente de dados. Vamos destrinchar o caso específico:

- O código `rand.unemp[J(all.dates), roll=0]` está dentro dos colchetes `[]` porque no `data.table` a operação básica é: `DT[i, j, by]` onde `i` serve para filtrar ou fazer join, `j` para selecionar ou calcular colunas e `by` para agrupamentos.

- Aqui, `J(all.dates)` é um atalho para `list(all.dates)`, que cria uma lista usada para o índice `i` (linhas). É a forma que o `data.table` entende como fazer join da tabela principal com as chaves da lista.

- O parâmetro `roll=0` não faz parte da função `J()`. Ele é passado como argumento do operador `[]` para controlar o comportamento do join, especificamente o tipo de "rolling join" a ser feito (neste caso, só aceita correspondência exata pois roll=0).

- Em resumo, o que ocorre é:  
  - Dentro do `[]`, o primeiro argumento `i` é `J(all.dates)`, que é a lista das chaves para o join.  
  - Fora do `J()`, mas ainda dentro dos `[]`, está o parâmetro `roll=0` que regula o comportamento da junção.  
  - O operador `[]` do `data.table` foi projetado para aceitar vários argumentos dessa forma (linha, coluna, opções join, etc.), diferente do formato usual de chamada de funções em R.

Essa separação permite expressar com clareza operações complexas em tabelas, como joins com parâmetros específicos, mantendo um estilo compacto e eficiente. Portanto, `J()` apenas cria a lista de chaves dentro de `i`, enquanto `roll=0` é um argumento para a operação de join via `[]` do `data.table`.

In [33]:
rand.unemp[, rpt := is.na(UNRATE)]

Cria uma coluna rpt em rand.unemp que indica se o valor da taxa de desemprego (UNRATE) está ausente (NA), identificando linhas com dados faltantes.